In [ ]:
!pip install torch transformers kobart
!pip install transformers datasets torch accelerate peft
!pip install -U bitsandbytes
!pip install newspaper3k
!pip install evaluate


ERROR: Could not find a version that satisfies the requirement kobart (from versions: none)
ERROR: No matching distribution found for kobart
  Preparing metadata (setup.py) ... done
  Created wheel for rogue: filename=rogue-0.0.2-py3-none-any.whl size=7209 sha256=fa1db0f80321214b18e0ed4e2c201ddc5c530c5cf4c688fa777ea01e2ea6d657
  Stored in directory: /root/.cache/pip/wheels/11/db/ee/a44bd5d88bd4903f31c1f677a28ac79e354555bca48332f232
Successfully built rogue


In [ ]:
!pip install newspaper3k

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import json
from typing import Dict
import json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

input_count = 0
data_to_save = []

# processed_dataset.jsonl 파일 읽기
with open('processed_dataset.jsonl', 'r', encoding='utf-8') as infile:
    for line in infile:
        # JSON 객체 로드
        data = json.loads(line)

        # input 개수 세기
        if 'input' in data:
            input_count += 1
            data_to_save.append(data)

        # 1000개가 되면 저장
        if input_count == 10000:
            with open('processed_cut10000_dataset.jsonl', 'w', encoding='utf-8') as outfile:
                for item in data_to_save:
                    outfile.write(json.dumps(item, ensure_ascii=False) + '\n')
            break

print(f"총 {input_count}개의 input이 처리되었습니다.")


총 10000개의 input이 처리되었습니다.


In [ ]:
import torch
from transformers import GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import json
import os
from transformers import PreTrainedTokenizerFast

# 하이퍼파라미터 설정
MODEL_NAME = 'skt/kogpt2-base-v2'
TRAIN_FILE = '/content/drive/MyDrive/ColabData/aisw/processed_dataset.jsonl'
OUTPUT_DIR = './kogpt2_news_summary_model'
BATCH_SIZE = 4
EPOCH = 3
LEARNING_RATE = 5e-5

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 토크나이저와 모델 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)

# 특수 토큰 추가 (필요한 경우)
tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 전처리 함수
def preprocess_dataset(file_path):
    inputs = []
    outputs = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            # 입력과 출력 텍스트 결합 (특수 토큰으로 구분)
            combined_text = f"{data['input']} </s> {data['output']}"
            inputs.append(combined_text)

    # 토큰화 및 인코딩
    encodings = tokenizer(inputs, truncation=True, max_length=512, padding='max_length')

    return encodings

# 데이터셋 준비
encodings = preprocess_dataset(TRAIN_FILE)

# 파이토치 데이터셋 생성
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone()
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# 데이터셋 초기화
dataset = NewsDataset(encodings)

# 트레이닝 아규먼트 설정
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCH,
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    evaluation_strategy='steps'
)

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    eval_dataset=dataset,)

# 모델 파인튜닝 시작
trainer.train()

# 모델 저장
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Fine-tuning 완료!")

# 간단한 추론 예제
def summarize_text(text, max_length=100):
    # 모델을 추론 모드로 설정
    model.eval()

    # 입력 텍스트 인코딩
    inputs = tokenizer.encode(text, return_tensors='pt').to(device)

    # 요약 생성
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # 요약 디코딩
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

Using device: cpu


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: eathan37 (anock7). Use `wandb login --relogin` to force relogin


In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset

# KoGPT-2 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# 데이터셋 로드 함수
def load_jsonl_dataset(file_path):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            dataset.append({
                'input': data['input'],
                'output': data['output']
            })
    return dataset

# 데이터 전처리 함수
def preprocess_function(examples, max_input_length=768, max_output_length=128):
    # 입력 텍스트 (신문 기사)와 출력 텍스트 (요약문)
    inputs = examples['input']
    outputs = examples['output']

    # 프롬프트 형식으로 텍스트 결합
    # 특정 포맷: "Article: [기사 내용]\nSummary: [요약문]"
    prompts = [
        f"Article: {input}\nSummary: {output}{tokenizer.eos_token}"
        for input, output in zip(inputs, outputs)
    ]

    # 토큰화 및 인코딩
    tokenized_inputs = tokenizer(
        prompts,
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # 레이블 설정 (GPT 모델의 경우 입력과 동일)
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].clone()

    return tokenized_inputs

# 데이터셋 로드 및 전처리 예시
def prepare_dataset(file_path):
    # 데이터셋 로드
    raw_dataset = load_jsonl_dataset(file_path)

    # Dataset 객체로 변환
    dataset = Dataset.from_list(raw_dataset)

    # 데이터셋 전처리
    processed_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names
    )

    return processed_dataset

# 모델 추론 함수
def generate_summary(model, tokenizer, text, max_length=128):
    # 프롬프트 형식 준비
    prompt = f"Article: {text}\nSummary:"

    # 토큰화
    inputs = tokenizer(
        prompt,
        return_tensors='pt',
        max_length=768,
        truncation=True
    )

    # 요약문 생성
    output_ids = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=4,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # 디코딩
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return summary

# 사용 예시
if __name__ == "__main__":
    # 데이터셋 경로 (실제 경로로 수정 필요)
    dataset_path = '/content/drive/MyDrive/ColabData/aisw/processed_dataset.jsonl'

    # 데이터셋 전처리
    processed_dataset = prepare_dataset(dataset_path)

    # 모델 로드
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # 첫 번째 샘플로 요약 테스트
    test_text = processed_dataset[0]['input']
    generated_summary = generate_summary(model, tokenizer, test_text)

    print("Original Article:", test_text)
    print("Generated Summary:", generated_summary)

Map:   0%|          | 0/23938 [00:00<?, ? examples/s]

KeyError: 'input'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 데이터셋 로드
def load_jsonl_dataset(file_path):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            dataset.append({
                'input': data['input'],
                'output': data['output']
            })
    return dataset

# 데이터셋 로드
raw_dataset = load_jsonl_dataset('/content/drive/MyDrive/ColabData/aisw/processed_dataset.jsonl')

# 데이터셋을 Hugging Face datasets 형식으로 변환
from datasets import Dataset
dataset = Dataset.from_list(raw_dataset)

# 학습/검증 데이터 분리 (90% 학습, 10% 검증)
dataset = dataset.train_test_split(test_size=0.1)

# KoBart 모델과 토크나이저 로드
model_name = "EbanLee/kobart-summary-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# 데이터 전처리 함수
def preprocess_function(examples):
    inputs = examples['input']
    targets = examples['output']

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # 타겟 텍스트 인코딩
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding='max_length'
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 전처리
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

training_args = Seq2SeqTrainingArguments(
    output_dir='./kobart_news_summary_model',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=500,
    save_total_limit=3,
    save_steps=500,
    predict_with_generate=True,
    fp16=True
)

# Trainer 초기화
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer
)

# 모델 학습
trainer.train()

# Google Drive에 모델 저장 경로 설정
google_drive_path = '/content/drive/My Drive/aisw/kobart_news_summary_model'

# 모델 저장
trainer.save_model(google_drive_path)
tokenizer.save_pretrained(google_drive_path)

# 모델 평가
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# 추론 예시
def summarize_text(text):
    inputs = tokenizer(
        text,
        max_length=512,
        return_tensors='pt',
        truncation=True
    ).to(device)

    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        max_length=128,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 모델 사용 예시
test_article = raw_dataset[0]['input']
generated_summary = summarize_text(test_article)
print("Original Article:", test_article[:500] + "...")
print("Generated Summary:", generated_summary)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,0.489000,0.463507
1000,0.486300,0.475107
1500,0.486500,0.472956
2000,0.503200,0.479312
2500,0.494900,0.480441
3000,0.378500,0.490719
3500,0.397900,0.493827
4000,0.402400,0.500114
4500,0.390900,0.493657
5000,0.398400,0.494427


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Evaluation Results: {'eval_loss': 0.6861209869384766, 'eval_runtime': 18.8501, 'eval_samples_per_second': 127.002, 'eval_steps_per_second': 15.915, 'epoch': 10.0}
Original Article: 제목: 여수광양항만공사, 광양항 배후단지‘투자유치’시동 본문: 중국 투자의향기업 직접 방문·투자협약 체결 협의 이성훈 sinawi@hanmail.net 여수광양항만공사(사장 방희석)는 14일부터 20일까지 광양항 배후단지 투자 유치를 위해 CEO가 직접 참여하는 투자유치 활동을 중국 중남부지역에서 펼친다. 광양만권경제자유구역청과 합동으로 진행되는 이번 투자유치활동은 방희석 사장이 직접 중국 현지의 투자의향기업을 방문해 광양항 및 배후단지의 장점 등을 소개하고 투자협약(MOU)을 체결하는 방식으로 진행된다. 방 사장은 먼저 광양항 서측배후단지 푸드존 투자유치를 위해 중국 운남성 소재 커피 원재료 공급업체인 운남허메이격치(주)를 방문해 광양항 배후단지 투자협약를 체결할 예정이다. 이어 한중일 콜드체인 거점 구축을 위해 중국 안후이성 소재 냉동냉장창고 업체인 바이란식품(주)을 방문해 투자협약을 맺기로 했다. 중국의 다른 입주의향기업들도 방문해 투자유치 활동을 진행할 계획이다. 이번 투자유...
Generated Summary: 여수광양항만공사는 광양만권경제자유구역청과 합동으로 직접 중국 현지의 투자의향기업을 방문해 광양항 배후단지 투자협약(MOU)을 체결하여 실질적인 투자가 이뤄지도록 활동을 진행한다.


In [ ]:
!pip install beautifulSoup4
!pip install newspaper3k
!pip install lxml[html_clean] lxml_html_clean


In [ ]:
from newspaper import Article
url = 'https://v.daum.net/v/20241208112020269'
article = Article(url, language='ko')
article.download()
article.parse()

In [ ]:
print('기사 제목 :')
print(article.title)
print('')

print('기사 내용 :')
print(article.text)

In [ ]:
# Google Drive에 저장된 모델과 tokenizer 로드
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
device = "cuda" if torch.cuda.is_available() else "cpu"

google_drive_path = '/content/drive/My Drive/aisw/kobart_news_summary_model'

# 모델과 tokenizer 로드
model = BartForConditionalGeneration.from_pretrained(google_drive_path).to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(google_drive_path)

# 텍스트 요약 함수
def summarize_text(text):
    inputs = tokenizer(
        text,
        max_length=512,
        return_tensors='pt',
        truncation=True
    ).to(device)

    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        max_length=128,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 요약문 생성 예시
test_article = article.text
generated_summary = summarize_text(test_article)

# 결과 출력
print("Original Article:", article.text[:100])
print("Generated Summary:", generated_summary)


In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 모델과 토크나이저 로드
model_name = "EbanLee/kobart-summary-v3"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 요약할 뉴스 기사
news_article = article.text

generated_summary = summarize_text(test_article)

print("Original Article:", article.text[:100])
print("Generated Summary:", generated_summary)



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Original Article: [편집자주] 윤석열 대통령의 계엄 사태에 정치적 불확실성이 커지면서 금융시장이 요동치고 있다. 원/달러 환율은 외국인 자금 이탈 우려에 원화 약세가 뚜렷해졌고 증시는 정부 정책 의
Generated Summary: 원/달러 환율은 원화 약세가 뚜렷해졌고 증시는 정부 정책 의존도가 높은 업종을 중심으로 투자 심리가 위축됐다.


In [ ]:
!pip install rouge

In [ ]:
import torch
import json
import evaluate
from datasets import Dataset
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from rouge import Rouge
from tqdm import tqdm

google_drive_path = '/content/drive/My Drive/aisw/kobart_news_summary_model'

device = "cuda" if torch.cuda.is_available() else "cpu"

def load_jsonl_dataset(file_path):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            dataset.append({
                'input': data['input'],
                'output': data['output']
            })
    return dataset

# 데이터셋 로드
raw_dataset = load_jsonl_dataset('/content/drive/MyDrive/ColabData/aisw/processed_dataset_valid.jsonl')
dataset = Dataset.from_list(raw_dataset)

# 모델과 tokenizer 로드
model = BartForConditionalGeneration.from_pretrained(google_drive_path).to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(google_drive_path)

# 텍스트 요약 함수
def summarize_text(text):
    inputs = tokenizer(
        text,
        max_length=512,
        return_tensors='pt',
        truncation=True
    ).to(device)

    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        max_length=128,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# ROUGE 초기화
rouge = Rouge()

# 전체 ROUGE 스코어를 저장할 리스트
total_rouge_scores = {
    'rouge-1': {'p': [], 'r': [], 'f': []},
    'rouge-2': {'p': [], 'r': [], 'f': []},
    'rouge-l': {'p': [], 'r': [], 'f': []}
}

# 무작위로 10개의 샘플 선택 (랜덤 시드 고정)
import random
random.seed(42)
sample_indices = random.sample(range(len(raw_dataset)), 10)

# 선택된 10개 샘플에 대해 ROUGE 스코어 계산
print("Validation Set ROUGE Scores for 10 Samples:")
for idx in sample_indices:
    # 원본 텍스트로부터 요약 생성
    input_text = raw_dataset[idx]['input']
    generated_summary = summarize_text(input_text)

    # 참조 요약문
    reference_summary = raw_dataset[idx]['output']

    # ROUGE 스코어 계산
    rouge_scores = rouge.get_scores(generated_summary, reference_summary)[0]

    print(f"\nSample {idx}:")
    print("Generated Summary:", generated_summary)
    print("Reference Summary:", reference_summary)

    # 각 ROUGE 메트릭별로 점수 저장
    for metric in ['rouge-1', 'rouge-2', 'rouge-l']:
        print(f"{metric.upper()}:")
        for score_type in ['p', 'r', 'f']:
            score = rouge_scores[metric][score_type]
            total_rouge_scores[metric][score_type].append(score)
            print(f"  {score_type.upper()}: {score:.4f}")

# 평균 ROUGE 스코어 계산
print("\nAverage ROUGE Scores across 10 Samples:")
avg_rouge_scores = {
    metric: {
        score_type: sum(scores) / len(scores)
        for score_type, scores in total_rouge_scores[metric].items()
    }
    for metric in ['rouge-1', 'rouge-2', 'rouge-l']
}

# 결과 출력
for metric, scores in avg_rouge_scores.items():
    print(f"{metric.upper()}:")
    for score_type, value in scores.items():
        print(f"  {score_type.upper()}: {value:.4f}")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Validation Set ROUGE Scores for 10 Samples:

Sample 2619:
Generated Summary: 한국거래소 유가증권시장본부는 4일 홍콩 포시즌스호텔에서 JP모간과 공동으로  '2019 대한민국 자본시장 라운드 테이블'을 개최하여 주주총회 내실화 방안과 기업지배구조 공시 가이드라인 등 최근 국내 정책동향을 전달하고, 한국 자본시장 발전을 위한 각종 제안내용을 청취하였으며, 향후 글로벌 커뮤니케이션 채널을 활용해 국내외 투자자들의 정책적 건의사항을 수렴하고, 필요한 경우 실제 정책에 반영될 수 있도록 적극 노력할 예정이라고 밝혔다.
Reference Summary: 한국거래소 유가증권시장본부는 4일 홍콩 포시즌스호텔에서 JP모간과 공동으로 '2019 대한민국 자본시장 라운드 테이블'을 주최했으며, 글로벌 30위 이내 초대형 투자기관과 연기금 등 20개사 고위 관계자가 참석한 가운데 국내 정책동향 및 한국 자본시장 발전을 위한 제안을 했다.
ROUGE-1:
  P: 0.3529
  R: 0.5455
  F: 0.4286
ROUGE-2:
  P: 0.2745
  R: 0.4242
  F: 0.3333
ROUGE-L:
  P: 0.3333
  R: 0.5152
  F: 0.4048

Sample 456:
Generated Summary: NS홈쇼핑은 방송심의와 소비자 관련 정책 및 제도에 대한 자문 및 개선 방안을 제시하는 '시청자위원회' 운영에 이어 건전한 방송을 만들고 소비자가 NS홈쇼핑의 정직하고 전문적인 방송을 믿고 시청할 수 있게 노력하고자 하는 의지로, 협력사와의 동반성장을 위해 품질관리와 공정한 상품 선정위원회 운영에 공을 들여 2019 고객감동브랜드지수 1위 홈쇼핑 부문을 3회 수상했다.
Reference Summary: 고객 신뢰를 바탕으로 고객 만족을 이끌어 내기 위해 끊임없이 제도를 마련하고 개선해온 NS홈쇼핑이  2019 고객감동브랜드지수 1위 홈쇼핑 부문을 3관왕 차지해 대한민국 최고의 유통사로 우뚝섰다.